In [1]:
from glob import glob as gg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import rc as rc
#
from netCDF4 import Dataset  

In [2]:
#
# mesh information
#
class mesh_data:
    def __init__(self, filename):
        self.read_vertical_mesh(filename)
        
    def read_vertical_mesh(self,filename):
        ncid = Dataset(filename, "r", format="NETCDF4")        
        
        # read dates 
        self.dates = ncid.variables['dates'][:]
        # read vertical mesh
        self.nlevels, self.zbar = ncid.variables['nlevels'][:], ncid.variables['zbar'][:]
        
        ncid.close()

In [3]:
#
# read mesh information
#
path_mesh = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/grid/'
filename = path_mesh + 'REcoM1D_mesh.nc'
mesh = mesh_data(filename)

In [4]:
#
# initial constant conditions
#
PAR_init = 50         # in W/m2
Kz_init = 0.01        # in m2/s
T_init = 1            # in degree Celsius
S_init = 34           # in ppt
aice_init = 90        # in %
uatm_init = 10        # in m/s
vatm_init = 10        # in m/s
press_init = 990*100  # in Pa
SW_init = 50   # in W/m2

In [18]:
#
# initialize arrays
#
nlvl, nt = len(mesh.zbar), len(mesh.nlevels)
# ice cover
aice = np.zeros(nt) + aice_init/100
# temperature, salinity, light and turbulence
init = np.zeros((nlvl-1, nt)) * np.nan
T, S, Kz = np.copy(init), np.copy(init), np.copy(init)
PAR = np.copy(init2D)
shortwave = np.zeros(nt)+SW_init
# atm forcing
uatm, vatm, sea_level_pressure = np.zeros(nt)+uatm_init, np.zeros(nt)+vatm_init, np.zeros(nt)+press_init

for i in range(nt):
    ind=int(mesh.nlevels[i])
    T[:ind,i], S[:ind,i], Kz[:ind,i] = T_init, S_init, Kz_init 
    PAR[:ind,i], shortwave[i] = PAR_init, SW_init

48 7201


In [20]:
#
# save forcing to netcdf
#
class Output(object):
    def __init__(self, filename, dates, aice, T, S, Kz, PAR, shortwave, uatm, vatm, sea_level_pressure):
        self.write_recom_constant_forcing(filename, dates, aice, T, S, Kz, PAR, shortwave, uatm, vatm, sea_level_pressure)
        
    def write_recom_constant_forcing(self, filename, dates, aice, T, S, Kz, PAR, shortwave, uatm, vatm, sea_level_pressure):
        
        nt, nl = len(dates), PAR.shape[0]
        
        # store grid (Lagrangian and vertical) information
        ncid = Dataset(filename, "w", format="NETCDF4") 

        ## define dimensionss
        ncid.createDimension('time', nt)       
        ncid.createDimension('level', nl)
        dim = ('level', 'time')

        ## create variables
        # dates
        dt = ncid.createVariable('dates', "f8",'time')
        
        # ice cover
        ice = ncid.createVariable('aice', "f8",'time')
                             
        # temperature and salinity
        t, s  = ncid.createVariable('temperature', "f8", dim), ncid.createVariable('salinity', "f8",dim)
        
        # atm forcing
        U, V = ncid.createVariable('uatm', "f8", 'time'), ncid.createVariable('vatm', "f8",'time')
        P = ncid.createVariable('sea_level_pressure', "f8", 'time')
        
        # vertical mesh
        par, kz  = ncid.createVariable('PAR', "f8",dim), ncid.createVariable('Kz', "f8",dim)
        sw = ncid.createVariable('surface_shortwave_radiation', "f8",'time')
                
        ## variables attributes
        # description
        dt.description='dates related to python ordinal dates (reference date 01/01/0001)'
        ice.description = 'surface of ocean covered by ice'
        t.description, s.description = 'temperature distribution in the water column',  'salinity distribution the water column'
        par.description, kz.description = 'Photosynthetically Active Radiation in the water column', 'turbulence in the water column'
        sw.description = 'surface shortwave radiation'
        U.description, V.description = 'x-component of wind', 'y-component of wind'
        P.description = 'mean sea level pressure'
        
        # units
        dt.units='days'
        ice.units, t.units, s.units = '', 'degree (Celsius)', 'ppt'
        par.units, sw.units, kz.units = 'W/m2', 'W/m2', 'm2/s'
        U.units, V.units, P.units = 'm/s', 'm/s', 'Pa'
        
        ## fill variables
        # dates
        ncid['dates'][:] = dates
        
        # ice
        ncid['aice'][:] = aice
        
        # temperature and salinity
        ncid['temperature'][:] , ncid['salinity'][:] = T, S
        
        # atm forcing
        ncid['uatm'][:], ncid['vatm'][:], ncid['sea_level_pressure'][:] = uatm, vatm, sea_level_pressure

        
        # radiation and turbulence
        ncid['PAR'][:], ncid['Kz'][:] = PAR, Kz
        ncid['surface_shortwave_radiation'][:] = shortwave
                             
        #
        ncid.close()
#
#
#
path_forcing = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/forcing/'
filename = path_forcing + 'REcoM1D_forcing.nc'
Output(filename, mesh.dates, aice, T, S, Kz, PAR, shortwave, uatm, vatm, sea_level_pressure)

('level', 'time') 46 7201 (47, 7201) (47, 7201)
